In [ ]:
print('Setup complete.')

# LAB 21 — Slack triage assistant

```


## Original CLI usage

```bash
bash
python lessons/day-08/labs/lab-21/main.py serve --port 8080
```


In [ ]:
# Environment check (adjust as needed)
import sys, platform
print(f'Python: {sys.version.split()[0]} on {platform.system()}')
# If imports fail, install requirements in your environment.
try:
    import httpx, jinja2, rich, typer, duckdb, pandas, matplotlib  # may be subset per lab
except Exception as e:
    print('Optional imports missing for some labs:', e)


In [ ]:
#!/usr/bin/env python3
"""LAB 21 — Slack triage assistant

Serve a minimal endpoint to handle /triage, summarize, and reply.
"""
# typer removed
from rich import print



def serve(port: int = 8080):
    print(f"[yellow]TODO:[/] Start a minimal HTTP server on :{port} to accept /triage.")
    print("[yellow]TODO:[/] Given a Slack thread link, fetch messages, summarize, and reply.")


In [ ]:
# Try the functions defined above. For example, if this lab had commands like `draft()` or `apply()`:
# draft()
# apply()
